In [28]:
# Section 1: CONFIGURATION

import pandas as pd
import numpy as np
import yfinance as yf

# 1. Universe and sector maps (fill these in when ready)
US_TICKERS = ['AAPL', 'MSFT', 'AMZN', 'JNJ', 'XOM']
IN_TICKERS = ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS']

SECTOR_MAP_US = {
    'AAPL': 'Information Technology',
    'MSFT': 'Information Technology',
    'AMZN': 'Consumer Discretionary',
    'JNJ':  'Health Care',
    'XOM':  'Energy'
}

SECTOR_MAP_IN = {
    'RELIANCE.NS':   'Energy',
    'TCS.NS':        'Information Technology',
    'HDFCBANK.NS':   'Financials',
    'INFY.NS':       'Information Technology',
    'ICICIBANK.NS':  'Financials'
}

# 2. Capital allocation
US_CAPITAL = 10_000       # USD
IN_CAPITAL = 1_000_000    # INR

# 3. Strategy parameters
LOOKBACK_MONTHS = 12      # months for momentum
SKIP_MONTHS     = 1       # skip most recent
STOP_LOSS       = 0.12    # 12% per-position stop-loss
REBALANCE_DAY   = 'MON'   # weekly on Mondays


In [29]:
# Section 2: DATA INGESTION 

def download_price_data(tickers, start_date, end_date):

    df = yf.download(tickers, start=start_date, end=end_date)
    
    # If multi-level columns (multiple tickers), pick the right field
    if isinstance(df.columns, pd.MultiIndex):
        if 'Adj Close' in df.columns.levels[0]:
            prices = df['Adj Close']
        else:
            prices = df['Close']
    else:
        # Single-level columns (e.g. one ticker)
        if 'Adj Close' in df.columns:
            prices = df['Adj Close']
        elif 'Close' in df.columns:
            prices = df['Close']
        else:
            raise KeyError("Neither 'Adj Close' nor 'Close' found in data")
    
    # Forward-fill missing days, then drop rows with all NaNs
    prices = prices.ffill().dropna(how='all')
    return prices

# Example usage (after populating US_TICKERS / IN_TICKERS):
start_date = '2015-01-01'
end_date   = '2025-06-06'

us_prices = download_price_data(US_TICKERS, start_date, end_date)
in_prices = download_price_data(IN_TICKERS, start_date, end_date)

print("US prices shape:", us_prices.shape)
print("IN prices shape:", in_prices.shape)


[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  5 of 5 completed

US prices shape: (2622, 5)
IN prices shape: (2573, 5)


In [30]:
#(**side note: this could be a good method while going forward with the tasks that we do, this way of going one-by-one like a boss would go over with an analyst and the progress checkpoint you make, lets make a llm wrapper ai task producitivty agent on the side) 

In [31]:
# Section 3: SIGNAL CALCULATION
def compute_momentum(prices: pd.DataFrame, lookback_months: int = 12, skip_months: int = 1) -> pd.DataFrame:
    
    # Approximate trading days per month
    days_per_month = 21
    
    # Calculate window and skip in trading days
    window = lookback_months * days_per_month
    skip = skip_months * days_per_month
    
    # 1. Compute percent change over (window + skip) days
    pct = prices.pct_change(periods=window + skip)
    
    # 2. Shift back by `skip` days so signal at time t uses t-(window+skip) to t-skip
    momentum = pct.shift(-skip)
    
    return momentum

In [32]:
def sector_parity_selection(signal: pd.Series, sector_map: dict, top_n: int = 25) -> list[str]:
    
    # Identify set of all sectors available in the current signal
    available_sectors = {sector_map[t] for t in signal.index if t in sector_map}
    unique_sector_count = len(available_sectors)

    picked = []
    used_sectors = set()

    # Iterate tickers by descending signal
    for ticker in signal.sort_values(ascending=False).index:
        if len(picked) >= top_n:
            break
        sector = sector_map.get(ticker)
        # 1) Pick one per sector until all unique sectors are covered
        if sector not in used_sectors:
            picked.append(ticker)
            used_sectors.add(sector)
        # 2) Once every sector has one pick, fill out to top_n by pure rank
        elif len(used_sectors) >= unique_sector_count:
            picked.append(ticker)

    return picked


In [33]:
import pandas as pd

# Section 5: Backtest Engine

def run_backtest(
    prices: pd.DataFrame,
    sector_map: dict,
    capital: float,
    lookback_months: int = 12,
    skip_months: int = 1,
    stop_loss: float = 0.12,
    freq: str = 'W-MON'
) -> (pd.Series, pd.DataFrame):

    # 1. Compute momentum signals once
    momentum = compute_momentum(prices, lookback_months, skip_months)
    
    # 2. Determine weekly rebalance dates (Monday close)
    weekly = prices.resample(freq).last()
    
    # 3. Prepare DataFrames for positions and NAV
    positions = pd.DataFrame(0.0, index=weekly.index, columns=prices.columns)
    
    # 4. Loop through each rebalance date
    for i in range(len(weekly.index) - 1):
        date = weekly.index[i]
        next_date = weekly.index[i + 1]
        
        # 4a. Extract the signal for 'date'
        if date not in momentum.index:
            # Carry forward any existing positions
            positions.loc[next_date] = positions.loc[date]
            continue
        
        signal = momentum.loc[date].dropna()
        
        # 4b. Select tickers based on sector-parity
        picks = sector_parity_selection(signal, sector_map, top_n=5)
        
        # 4c. Assign equal weight to each pick
        weight = 1.0 / len(picks)
        positions.loc[date, :] = 0.0
        positions.loc[date, picks] = weight
        
        # 4d. Carry positions forward until next rebalance
        positions.loc[next_date] = positions.loc[date]
        
        # 4e. Apply per-position stop-loss intra-week
        week_prices = prices.loc[date:next_date]
        returns = week_prices / week_prices.iloc[0] - 1.0
        stop_mask = returns.min() < -stop_loss
        for ticker in prices.columns[stop_mask]:
            positions.loc[next_date, ticker] = 0.0
    
    # 5. Compute NAV over time
    daily_value = (positions.shift() * prices).sum(axis=1)
    nav = daily_value * (capital / daily_value.iloc[0])
    
    return nav, positions
